Cell 1 — Imports & Config

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

Cell 2 — Positional Encoding

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)

        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

Cell 3 — Scaled Dot-Product Attention

In [4]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    d_k = Q.size(-1)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    attention = torch.softmax(scores, dim=-1)
    output = torch.matmul(attention, V)
    return output, attention

Cell 4 — Multi-Head Attention

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0

        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)

        self.fc = nn.Linear(d_model, d_model)

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)

        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        output, attention = scaled_dot_product_attention(Q, K, V, mask)

        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, -1, self.num_heads * self.d_k)

        return self.fc(output)

Cell 5 — Feed Forward Network

In [6]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

Cell 6 — Transformer Encoder Block

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        x = self.norm1(x + self.attn(x, x, x, mask))
        x = self.norm2(x + self.ff(x))
        return x


Cell 7 — Transformer Encoder

In [8]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        x = self.embedding(x)
        x = self.positional(x)

        for layer in self.layers:
            x = layer(x, mask)

        return x

Cell 8 — Test Forward Pass

In [9]:
VOCAB_SIZE = 10000
D_MODEL = 128
NUM_LAYERS = 2
NUM_HEADS = 4
D_FF = 256

model = TransformerEncoder(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    num_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    d_ff=D_FF
)

x = torch.randint(0, VOCAB_SIZE, (32, 20))
out = model(x)

print("Output shape:", out.shape)

Output shape: torch.Size([32, 20, 128])


Cell 9 — Masks (Padding & Look-Ahead

In [10]:
def create_padding_mask(seq, pad_idx=0):
    # (batch, 1, 1, seq_len)
    return (seq != pad_idx).unsqueeze(1).unsqueeze(2)

def create_look_ahead_mask(size):
    # (1, 1, size, size)
    mask = torch.tril(torch.ones(size, size))
    return mask.unsqueeze(0).unsqueeze(1)

Cell 10 — Decoder Layer (Masked Self-Attention)

In [11]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_attn = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, x, enc_out, look_ahead_mask=None, padding_mask=None):
        # Masked self-attention
        x = self.norm1(x + self.self_attn(x, x, x, look_ahead_mask))
        # Encoder–Decoder attention
        x = self.norm2(x + self.enc_attn(x, enc_out, enc_out, padding_mask))
        # Feed-forward
        x = self.norm3(x + self.ff(x))
        return x


Cell 11 — Transformer Decoder

In [12]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x, enc_out, look_ahead_mask=None, padding_mask=None):
        x = self.embedding(x)
        x = self.positional(x)

        for layer in self.layers:
            x = layer(x, enc_out, look_ahead_mask, padding_mask)

        return self.fc(x)

Cell 12 — Full Transformer (Encoder + Decoder

In [13]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff):
        super().__init__()
        self.encoder = TransformerEncoder(vocab_size, d_model, num_layers, num_heads, d_ff)
        self.decoder = TransformerDecoder(vocab_size, d_model, num_layers, num_heads, d_ff)

    def forward(self, src, tgt, src_mask, tgt_mask):
        enc_out = self.encoder(src, src_mask)
        out = self.decoder(tgt, enc_out, tgt_mask, src_mask)
        return out

Cell 13 — Quick Forward Test

In [14]:
VOCAB_SIZE = 10000
D_MODEL = 128
NUM_LAYERS = 2
NUM_HEADS = 4
D_FF = 256

model = Transformer(VOCAB_SIZE, D_MODEL, NUM_LAYERS, NUM_HEADS, D_FF)

src = torch.randint(0, VOCAB_SIZE, (32, 20))
tgt = torch.randint(0, VOCAB_SIZE, (32, 20))

src_mask = create_padding_mask(src)
tgt_mask = create_look_ahead_mask(tgt.size(1))

out = model(src, tgt, src_mask, tgt_mask)
print("Transformer output shape:", out.shape)

Transformer output shape: torch.Size([32, 20, 10000])


Cell 14 — Training Loop (Skeleton)

In [15]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def train_step(src, tgt):
    model.train()

    tgt_input = tgt[:, :-1]
    tgt_real = tgt[:, 1:]

    src_mask = create_padding_mask(src)
    tgt_mask = create_look_ahead_mask(tgt_input.size(1))

    preds = model(src, tgt_input, src_mask, tgt_mask)

    loss = criterion(
        preds.reshape(-1, VOCAB_SIZE),
        tgt_real.reshape(-1)
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

Cell 15 — Greedy Decoding (Inference)

In [16]:
def greedy_decode(model, src, max_len=20, sos_idx=2, eos_idx=3):
    model.eval()

    src_mask = create_padding_mask(src)
    enc_out = model.encoder(src, src_mask)

    tgt = torch.tensor([[sos_idx]])

    for _ in range(max_len):
        tgt_mask = create_look_ahead_mask(tgt.size(1))
        out = model.decoder(tgt, enc_out, tgt_mask, src_mask)

        next_token = out[:, -1].argmax(dim=-1).item()
        tgt = torch.cat([tgt, torch.tensor([[next_token]])], dim=1)

        if next_token == eos_idx:
            break

    return tgt